<a href="https://colab.research.google.com/github/jblancoperez/fiap-desafio/blob/main/C%C3%B3pia_de_00_analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import plot_confusion_matrix



from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR




#!{sys.executable} -m pip install -U pandas-profiling[notebook]
#!jupyter nbextension enable --py widgetsnbextension

#!{sys.executable} -m pip install -U lafrom lazypredict.Supervised import LazyClassifier, LazyRegressor

def cm_to_inch(value):
    return value/2.54
#arquivo = pd.read_csv('https://raw.githubusercontent.com/jblancoperez/fiap-desafio/main/solicitacoescredito.csv')

In [13]:
arquivo = pd.read_csv('./solicitacoescredito.csv')
aprovadosAnalista=arquivo.loc[arquivo['status'].isin(['AprovadoAnalista','AprovadoComite'])]
aprovadosAnalista.head()








,numero_solicitacao,razaoSocial,nomeFantasia,cnpjSemTraco,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,scorePontualidade,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista
0,1,James Richardson-Patel,Alexandra Williams,KEBE17609492220843,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,...,12.0,0.0,2003.0,Acima de 17 anos,90000.0,False,True,1.0,43200.0,2020-02-03T20:57:33
2,3,Joanna Hudson,Dr. David Rees,GCVQ28531614261293,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,...,7.0,0.0,2014.0,De 6 a 10 anos,20000.0,False,True,1.0,4320.0,2020-02-04T16:40:49
3,4,Gordon Jones-Hopkins,Sara Reid-Robson,KJND32266018316396,20,0.000000,NaN,2017-02-13T17:20:27,0,0.00,...,12.0,0.0,2013.0,De 6 a 10 anos,30000.0,False,True,0.0,5920.0,2020-02-04T16:37:52
4,5,Nigel Lee,Dr. Stanley Duncan,CGQN15826802440348,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,1.0,89000.0,2020-02-04T15:06:28
5,6,Liam Jackson,Gerard Gordon,KAYS53232027306925,0,0.000000,NaN,2019-06-28T10:25:57,0,0.00,...,12.0,0.0,2011.0,De 6 a 10 anos,15000.0,False,True,0.0,38400.0,2020-02-04T15:10:46


In [14]:
#Feature selection class to eliminate multicollinearity

#Baseado em  https://www.youtube.com/watch?v=ioXKxulmwVQ&feature=youtu.be
class MultiCollinearityEliminator():
    
    #Class Constructor
    def __init__(self, df, target, threshold):
        self.df = df
        self.target = target
        self.threshold = threshold

    #Method to create and return the feature correlation matrix dataframe
    def createCorrMatrix(self, include_target = False):
        #Checking we should include the target in the correlation matrix
        if (include_target == False):
            df_temp = self.df.drop([self.target], axis =1)
            
            #Setting method to Pearson to prevent issues in case the default method for df.corr() gets changed
            #Setting min_period to 30 for the sample size to be statistically significant (normal) according to 
            #central limit theorem
            corrMatrix = df_temp.corr(method='pearson', min_periods=30).abs()
        #Target is included for creating the series of feature to target correlation - Please refer the notes under the 
        #print statement to understand why we create the series of feature to target correlation
        elif (include_target == True):
            corrMatrix = self.df.corr(method='pearson', min_periods=30).abs()
        return corrMatrix

    #Method to create and return the feature to target correlation matrix dataframe
    def createCorrMatrixWithTarget(self):
        #After obtaining the list of correlated features, this method will help to view which variables 
        #(in the list of correlated features) are least correlated with the target
        #This way, out the list of correlated features, we can ensure to elimate the feature that is 
        #least correlated with the target
        #This not only helps to sustain the predictive power of the model but also helps in reducing model complexity
        
        #Obtaining the correlation matrix of the dataframe (along with the target)
        corrMatrix = self.createCorrMatrix(include_target = True)                           
        #Creating the required dataframe, then dropping the target row 
        #and sorting by the value of correlation with target (in asceding order)
        corrWithTarget = pd.DataFrame(corrMatrix.loc[:,self.target]).drop([self.target], axis = 0).sort_values(by = self.target)                    
        print(corrWithTarget, '\n')
        return corrWithTarget

    #Method to create and return the list of correlated features
    def createCorrelatedFeaturesList(self):
        #Obtaining the correlation matrix of the dataframe (without the target)
        corrMatrix = self.createCorrMatrix(include_target = False)                          
        colCorr = []
        #Iterating through the columns of the correlation matrix dataframe
        for column in corrMatrix.columns:
            #Iterating through the values (row wise) of the correlation matrix dataframe
            for idx, row in corrMatrix.iterrows():                                            
                if(row[column]>self.threshold) and (row[column]<1):
                    #Adding the features that are not already in the list of correlated features
                    if (idx not in colCorr):
                        colCorr.append(idx)
                    if (column not in colCorr):
                        colCorr.append(column)
        print(colCorr, '\n')
        return colCorr

    #Method to eliminate the least important features from the list of correlated features
    def deleteFeatures(self, colCorr):
        #Obtaining the feature to target correlation matrix dataframe
        corrWithTarget = self.createCorrMatrixWithTarget()                                  
        for idx, row in corrWithTarget.iterrows():
            print(idx, '\n')
            if (idx in colCorr):
                self.df = self.df.drop(idx, axis =1)
                break
        return self.df

    #Method to run automatically eliminate multicollinearity
    def autoEliminateMulticollinearity(self):
        #Obtaining the list of correlated features
        colCorr = self.createCorrelatedFeaturesList()                                       
        while colCorr != []:
            #Obtaining the dataframe after deleting the feature (from the list of correlated features) 
            #that is least correlated with the taregt
            self.df = self.deleteFeatures(colCorr)
            #Obtaining the list of correlated features
            colCorr = self.createCorrelatedFeaturesList()                                     
        return self.df

In [41]:
#Remove MultiCollinearity in input
arquivo = pd.read_csv('./solicitacoescredito.csv')
df=arquivo.loc[arquivo['status'].isin(['AprovadoAnalista','AprovadoComite'])]
df=df.drop(['numero_solicitacao','razaoSocial','nomeFantasia','cnpjSemTraco','dataAprovadoEmComite','dataAprovadoNivelAnalista'],axis=1)


invalidDates = [
    '0019-02-06T03:06:00',
    '0001-01-01T03:06:00',
    '0001-01-01T06:12:00',
    '0019-02-06T03:06:00',
    '0219-12-31T03:06:00'
]

for d in invalidDates:
    df['periodoBalanco']=df['periodoBalanco'].replace(d,np.nan)
df['periodoBalanco']=pd.to_datetime(df['periodoBalanco'],errors='coerce')
df['primeiraCompra']=pd.to_datetime(df['primeiraCompra'],errors='coerce')


df['eprimeira']=df['primeiraCompra'].isna()
df=df.drop(['primeiraCompra'],axis=1)

ftRemover = MultiCollinearityEliminator(df,'valorAprovado',0.1)
df=ftRemover.autoEliminateMulticollinearity()
df=df.drop(['periodoBalanco'],axis=1)
df.info()
df=pd.get_dummies(df, columns=["definicaoRisco","empresa_MeEppMei","restricoes","intervaloFundacao","status"], prefix=["definicaoRisco","mei","restricoes","intervaloFundacao","status"])
#Create dummy columns
df=df.fillna(method='bfill',axis=1)
y = df['valorAprovado']
X= df.drop(['valorAprovado'], axis=1)
X = StandardScaler().fit_transform(X)


['prazoMedioRecebimentoVendas', 'margemBrutaAcumulada', 'titulosEmAberto', 'endividamento', 'estoque', 'faturamentoBruto', 'scorePontualidade', 'eprimeira', 'periodoDemonstrativoEmMeses', 'capitalSocial', 'limiteEmpresaAnaliseCredito', 'dashboardCorrelacao', 'diferencaPercentualRisco', 'anoFundacao', 'percentualRisco', 'passivoCirculante', 'ativoCirculante', 'totalAtivo', 'totalPatrimonioLiquido', 'duplicatasAReceber', 'margemBruta', 'custos'] 

                             valorAprovado
percentualProtestos               0.008198
eprimeira                         0.014801
dashboardCorrelacao               0.030369
valorSolicitado                   0.048894
duplicatasAReceber                0.049847
anoFundacao                       0.052694
passivoCirculante                 0.056105
margemBrutaAcumulada              0.059794
diferencaPercentualRisco          0.060300
percentualRisco                   0.060300
maiorAtraso                       0.067067
totalAtivo                        

In [ ]:
#Create dummy columns
df=df.drop(['numero_solicitacao','razaoSocial','nomeFantasia','cnpjSemTraco','dataAprovadoEmComite','dataAprovadoNivelAnalista'],axis=1)
df=pd.get_dummies(df, columns=["status"],prefix=["status"])
df.info()
df[df.isnull().any(axis=1)]
df=df.fillna(method='bfill',axis=1)
y = df['valorAprovado']
X= df.drop(['valorAprovado'], axis=1)
X = StandardScaler().fit_transform(X)

In [49]:
names = ["Nearest Neighbors", 
         "Linear SVM", 
         "RBF SVM", 
         #"Gaussian Process",
         #"Decision Tree", 
         #"Random Forest", 
         #"Neural Net", 
         "AdaBoost",
         #"Naive Bayes", 
         #"QDA"
        ]

classifiers = [
    KNeighborsRegressor(3),
    SVR(kernel="linear", C=0.025),
    #SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    #DecisionTreeClassifier(max_depth=5),
    #RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    #MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostRegressor(),
    #QuadraticDiscriminantAnalysis()
    ]
l=len(classifiers)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)




from sklearn.model_selection import cross_val_score


for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = cross_val_score(clf,X,y,scoring = "neg_mean_squared_error")

    print (name,score)
  
    i=1
    
   




Nearest Neighbors [-1.87017739e+11 -7.88425261e+10 -7.08713715e+10 -1.50520975e+11
 -2.91328982e+11]
Linear SVM [-1.20622700e+11 -1.48619834e+11 -1.80182356e+11 -5.31916791e+11
 -6.13638920e+11]
RBF SVM [-6.45145058e+10 -3.42742272e+10 -1.89090076e+10 -5.65929770e+10
 -5.96278611e+10]


In [50]:
df=df.fillna(method='bfill',axis=1)

X= df.drop(['credito'], axis=1)
X = StandardScaler().fit_transform(X)


names = ["Nearest Neighbors", 
         "Linear SVM", 
         "RBF SVM", 
         #"Gaussian Process",
         #"Decision Tree", 
         #"Random Forest", 
         #"Neural Net", 
         #"AdaBoost",
         "Naive Bayes", 
         "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    #SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    #DecisionTreeClassifier(max_depth=5),
    #RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    #MLPClassifier(alpha=1, max_iter=1000),
    #AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
l=len(classifiers)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

class_names=['Rejeitado', 'Concedido']
plt.figure(figsize=(cm_to_inch(25), cm_to_inch(15)))

import sklearn.metrics

for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print (name,score)
  
    i=1
    
    titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
    for title, normalize in titles_options:
        i+=1
        disp = plot_confusion_matrix(clf, X_test, y_test,
                                 display_labels=class_names,
                                 normalize=normalize)

#        accuracy_score = metrics.accuracy_score(y_test, y_pred)
#        precision_score = metrics.precision_score(y_test, y_pred)
#        metrics.recall_score = metrics.recall_score(y_test, y_pred)
#        metrics.f1_score = metrics.f1_score(y_test, y_pred)
        plt.show()    
    


KeyError: "['credito'] not found in axis"